# Transcribo

#####An ML based program which transcribes Online video lectures/meetings and adds highlights/topics generated and summaries of the video lecture.



In [1]:
#imports
!pip install moviepy pydub SpeechRecognition gdown
!pip install punctuator 
import gdown
import numpy as np
import speech_recognition as sr 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import wave, math, contextlib
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import warnings
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from moviepy.editor import AudioFileClip
nltk.download('stopwords')
try :
  nltk.data.find('tokenizers/punkt')
  
except LookupError:
  print('punkt')
  nltk.download('punkt')

try :
  nltk.data.find('corpora/wordnet')
except LookupError:
  nltk.download('wordnet')
print("Imports Done")

C:\Users\avdhu\miniconda3\envs\transcribo\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Imports Done


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avdhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Converting the Video file to Audio Wave
audioclip = AudioFileClip("exa.mp4")
audioclip.write_audiofile("file.wav")

chunk:   0%|                                                                       | 0/12112 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in file.wav


MoviePy - Done.


In [7]:
#Getting the necessary features of audio file for Speech Recognition
with contextlib.closing(wave.open("file.wav",'r')) as f:
    waves = f.getnframes()
    framerate = f.getframerate()
    duration = waves / float(framerate)
finaltotal = math.ceil(duration / 60)

In [10]:
#Speech recognition in the audio file
text=''
r = sr.Recognizer()
for i in range(0, finaltotal):
    with sr.AudioFile("file.wav") as source:
        audio = r.record(source, offset=i*60, duration=60)
    f = open("transcription.txt", "a")
    f.write(r.recognize_google(audio))
    f.write(" ")
    print(i,"and",finaltotal)
f.close()


0 and 10
1 and 10
2 and 10
3 and 10
4 and 10
5 and 10
6 and 10
7 and 10
8 and 10
9 and 10


In [4]:
#Punctuating the raw transcript 
text=open("transcription.txt", "r")
text=text.read()
from punctuator import Punctuator

p = Punctuator('hel.pcl')
punctuated=p.punctuate(text)
print(punctuated)

What is the industry by staying with you that I wish I'd known before I started reading a few other youtubers Who done on this topic and I want to put my own twist on things in this video, the five things I will be sharing of things you've never heard of before or at least equal. Then it usually involved in 16 minutes, I want to switch things up that Hill. You know, my middle name is the experimenter clients. We experimentally harvesting in the spirit would not mind emename I want an experiment make. This is very short and sweet bitesize. It will keep it under 5 minutes forward I went over 5 minutes. Ok. So the first thing I wish, I'd known before I started coding. And this is the one of the four it's always good to hear again is that there is no after which it too late to start Oracle right now watching is very when you want to start to create a great time to start. If you study it college in you are you never coronavirus the foreign? Can you don't worry, it's, a great time to start i

In [5]:
#Working through the summary
#We did delibrately added punctuations, but for vectorizing the text, we need to get them out too.
HANDICAP=1
def remove_punctuation_marks(text) :
    punctuation_marks = dict((ord(punctuation_mark), None) for punctuation_mark in string.punctuation)
    return text.translate(punctuation_marks)
#Tokenizing the words using nltk
def get_lemmatized_tokens(text) :
    normalized_tokens = nltk.word_tokenize(remove_punctuation_marks(text.lower()))
    return [nltk.stem.WordNetLemmatizer().lemmatize(normalized_token) for normalized_token in normalized_tokens]
#Getting average of the values of the sentences scored
def get_average(values) :
    greater_than_zero_count = total = 0
    for value in values :
        if value != 0 :
            greater_than_zero_count += 1
            total += value 
    return total / 1
#Getting the final score of the sentences/words.
def get_threshold(tfidf_results) :
    i = total = 0
    while i < (tfidf_results.shape[0]) :
        total += get_average(tfidf_results[i, :].toarray()[0])
        i += 1
    return total / tfidf_results.shape[0]
#Getting the final summary with the most scored sentences in text
def get_summary(documents, tfidf_results) :
    summary = ""
    i = 0
    while i < (tfidf_results.shape[0]) :
        if (get_average(tfidf_results[i, :].toarray()[0])) >= get_threshold(tfidf_results) * HANDICAP :
                summary += ' ' + documents[i]
        i += 1
    return summary

if __name__ == "__main__" :
    warnings.filterwarnings("ignore")
    documents = nltk.sent_tokenize(punctuated)
    tfidf_results = TfidfVectorizer(tokenizer = get_lemmatized_tokens, stop_words = stopwords.words('english')).fit_transform(documents)
    a=get_summary(documents, tfidf_results)
    print(a)
    print(len(punctuated))

 What is the industry by staying with you that I wish I'd known before I started reading a few other youtubers Who done on this topic and I want to put my own twist on things in this video, the five things I will be sharing of things you've never heard of before or at least equal. And this is the one of the four it's always good to hear again is that there is no after which it too late to start Oracle right now watching is very when you want to start to create a great time to start. Can you don't worry, it's, a great time to start if you feel that if you just garbage next mein to dispel the nest for you right now, if you're just starting out coding 39 operating systems, we can take from Windows to Mac OS and some variation of Linux Ubuntu. For example, you can use whatever you want whatever what is most convenient to you and a fact that all three of them have a small town V right now. The downside is that most coding, tutorials online, Auto, Tech done on max on Linux machine circuit or

In [3]:
#Getting the keywords out of the text, with gensim's summarization tool
from gensim.summarization import keywords
text=open("transcription.txt", "r")
text=text.read()
text_en = text
final=keywords(text_en,words = 10, lemmatize = True)
print(final)
type(final)

code
developers
programming
software
engineer
number
thing
youtube
pisces
usually


str

In [20]:
f=open("Final Transcription.txt",'w')
f.write("Transcription for Given Source.\n")
f.write("\nThe following keywords were extracted:\n")
f.write(final)
f.write("\nThe summarized Block of the transcription: \n")
f.write(a)
f.write("\nThe original transcription:\n")
f.write(punctuated)
f.close
print("done")

done
